In [28]:
import numpy as np
import numpy as cp
import pandas as pd
from scipy.sparse import csr_matrix, kron
from scipy.sparse import identity
import scipy.sparse as sp
import random
from scipy import linalg
from IPython.display import clear_output

In [31]:
def calculate_smoothed_weights(mat):
    identity_matrix = np.identity(mat.shape[0])
    W_smooth = -pd.DataFrame(identity_matrix).rolling(2, axis=1).sum().replace(np.nan, 0)
    W_smooth[0] = -pd.DataFrame(identity_matrix)[0]
    W_smooth = np.array(W_smooth + 2 * pd.DataFrame(identity_matrix))
    W_smooth_pos = (np.abs(W_smooth) + W_smooth) / 2
    W_smooth_neg = (np.abs(W_smooth) - W_smooth) / 2
    neum_term_B = cp.asarray(W_smooth_neg.T.dot(W_smooth_neg))
    denom_term_B = cp.asarray(W_smooth_pos.T.dot(W_smooth_pos))
    return  (csr_matrix(neum_term_B), csr_matrix(denom_term_B))
def calculate_rmse(y_true, y_pred):
    squared_diff = (y_true - y_pred) ** 2
    mean_squared_diff = cp.mean(squared_diff)
    rmse = cp.sqrt(mean_squared_diff)
    return rmse.item()

In [32]:
import pandas as pd
import numpy as np

# Create a 96x96 DataFrame filled with -1
df = pd.DataFrame(np.full((96, 96), 0))

# Set diagonals as 0
for i in range(96):
    df.iat[i, i] = 0
for i in range(96):
    for j in range(max(0, i-30), min(96, i+30)):
        if i != j:
            df.iat[i, j] = 0
            df.iat[j, i] = 0
# Set the four next and four previous cells (in rows and columns) to each diagonal cell as 1
for i in range(96):
    for j in range(max(0, i-2), min(96, i+4)):
        if i != j:
            df.iat[i, j] = 1
            df.iat[j, i] = 1
ad = df.copy(deep = True)
ad = (ad+ad.T)/2
deg = pd.DataFrame(np.diag(ad.sum(axis = 0)), index = ad.index, columns = ad.columns)
ad = csr_matrix(np.array(ad))
deg = csr_matrix(np.array(deg))

In [44]:

def GRU_NMF(array1, ad, deg, r, hy2, alpha_B, smoothening_hyp,n_iter = 4000, prin = 100, time_const= 1):
    A = csr_matrix(np.random.rand(array1.shape[0], r)).astype('float32')
    B = csr_matrix(np.random.rand(array1.shape[1], r)).astype('float32')

    neum_term_B, denom_term_B = calculate_smoothed_weights(B)

    for iteration in range(n_iter):
        #print(type (B))
        time_neum = time_const * (ad.dot(B))
        time_denom = time_const * (deg.dot(B))
        A_neum = array1.dot(B)
        A_denom = A.dot(B.T).dot(B) + hy2 * A
        A = A.multiply((A_neum) / (A_denom))
        A.data = np.nan_to_num(A.data)

        B_neum = array1.T.dot(A) + (alpha_B * (B.T)).T + smoothening_hyp * (neum_term_B.dot(B)) + time_neum
        B_denom = B.dot(A.T).dot(A) + hy2 * B + (alpha_B * (B.T.dot(B)).dot(B.T)).T + smoothening_hyp * (denom_term_B.dot(B)) + time_denom
        B = B.multiply((B_neum) / (B_denom))
        B.data = np.nan_to_num(B.data)

        
        if iteration % prin == 0:
            rmse = calculate_rmse(array1.toarray(), A.dot(B.T).toarray())
            ortho_A = ((Identity - B.tocsr().T.dot(B.tocsr())) * (Identity - B.tocsr().T.dot(B.tocsr()))).sum()
            time_reg = (B.T.dot(deg - ad).dot(B)).sum()
            print("Iteration", iteration, "RMSE:", rmse, time_reg, ortho_A)

    return A, B